In [ ]:
# Auto-detect GPUs, set env, and launch the HPO script from Jupyter

import os, sys, subprocess, shlex, json, pathlib

PROJECT_DIR = "/work/projects/myproj"            # change if needed
SCRIPT      = "train_hpo_both_static_ovr.py"     # the file you pasted earlier

# ---- GPU detection (respect CUDA_VISIBLE_DEVICES if already set) ----
def detect_gpu_count():
    # If the user already masked GPUs, respect that
    vis = os.environ.get("CUDA_VISIBLE_DEVICES", "").strip()
    if vis:
        return len([x for x in vis.split(",") if x.strip() != ""])
    # Try torch
    try:
        import torch
        return int(torch.cuda.device_count())
    except Exception:
        pass
    # Fallback: nvidia-smi
    try:
        out = subprocess.check_output(["nvidia-smi", "-L"], stderr=subprocess.DEVNULL, text=True)
        return len([ln for ln in out.splitlines() if "GPU " in ln])
    except Exception:
        return 0

gpu_count = detect_gpu_count()
if gpu_count == 0:
    print("⚠️ No GPUs detected. The script will run CPU-only or fail if CUDA is required.")
else:
    print(f"✅ Detected {gpu_count} GPU(s)")

# Let the script decide mapping; just hint max parallel = number of visible GPUs
os.environ.setdefault("MAX_PARALLEL", str(max(1, gpu_count)))
# allocator safety
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")
# Optional perf toggles (script also sets them per-worker)
os.environ.setdefault("TF32", "1")

# ---- Optional helpful env (edit if your paths differ) ----
# These are only used if your .py doesn’t already find them via defaults.
# os.environ.setdefault("STAT_DATASETS_ROOT", f"{PROJECT_DIR}/SOLO_Supervised_RFDETR/Stat_Dataset")
# os.environ.setdefault("IMAGES_FALLBACK_ROOT", "/work/Member Files:yourname/CellScanData/Zoom10x - Quality Assessment_Cleaned")
# os.environ.setdefault("OUTPUT_ROOT", "/work/Member Files:yourname/RFDETR_SOLO_OUTPUT/HPO_BOTH_OVR")
# os.environ.setdefault("NUM_WORKERS", "8")  # dataloader workers

# ---- Run the script ----
wd = pathlib.Path(PROJECT_DIR).resolve()
py = sys.executable
cmd = f'{shlex.quote(py)} -u {shlex.quote(str(wd / SCRIPT))}'

print("\n[LAUNCH]")
print(" cwd:", wd)
print(" cmd:", cmd)
print(" env: MAX_PARALLEL=", os.environ.get("MAX_PARALLEL"))

proc = subprocess.Popen(cmd, cwd=str(wd), shell=True)
# If you prefer to block and see output inline, use:
# subprocess.run(cmd, cwd=str(wd), shell=True, check=True)
